## Online Simulation-Based Digital Twin: Nibelungenbrücke

This notebook introduces an online simulation-based digital twin developed for the Nibelungenbrücke bridge. The orchestration system is designed to interact dynamically with the user by gathering key inputs; such as time steps, physical parameters, and spatial positions to perform real-time simulations of the bridge structure.

The simulations are powered by the FenicSXConcrete package and support both thermomechanical and structural deflection analyses. This framework enables realistic, physics-based modeling and continuous digital monitoring of the bridge's behavior.

Note: 

After confirming that you're operating within the correct Python environment, if the model still raises any library-related errors, activate the following command to ensure that all required packages are properly installed!

In [1]:

#import sys
#!{sys.executable} -m pip install git+https://github.com/BAMresearch/FenicsXConcrete pint gmsh pytest jsonschema pandas pyproj tqdm pvlib


Import the necessary modules and the `DigitalTwinInterface` class from the project structure.

In [2]:
import os
import random
from nibelungenbruecke.scripts.digital_twin_orchestrator.interface_test_DT import DigitalTwinInterface

# Configuration Path
Set the path to the JSON file that holds the default digital twin parameters.
-> You may adjust the path if you're running the notebook from a different directory.

# Digital Twin Initialization
Create an instance of the digital twin interface using the provided configuration file.


In [3]:
path = 'use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json'
# path = '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json'

dt = DigitalTwinInterface(path)


FileNotFoundError: Error: The file ../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json was not found!

Execute the digital twin simulation using the loaded model and parameters. This will simulate the bridge behavior based on the selected physics model (e.g., thermal or displacement) and input configuration.

In [ ]:
dt.run_model()